In [1]:
%%writefile assign2.cu

#include <stdio.h>
#define N 5

// Define a static_global memory array

__device__ int Aglobal[N];
__device__ int Bglobal[N];
__device__ int Cglobal[N];

// Kernel function to modify the static global memory
__global__ void addArray(){
  int idx = threadIdx.x;

  if (idx < N){
  Cglobal[idx] = Aglobal[idx] + Bglobal[idx];
  }
}


void initializeGlobal_A(int *hostArray){
//  cudaMemcpyToSymbol(Aglobal, hostArray, N*sizeof(int), 0, cudaMemcpyHostToDevice);

   cudaMemcpyToSymbol(Aglobal, hostArray, N*sizeof(int));
}

void initializeGlobal_B(int *hostArray){
//  cudaMemcpyToSymbol(Bglobal, hostArray, N*sizeof(int), 0, cudaMemcpyHostToDevice);

   cudaMemcpyToSymbol(Bglobal, hostArray, N*sizeof(int));
}


void retrieveGlobalArray(int *hostArray){
//  cudaMemcpyFromSymbol(hostArray, Cglobal, N*sizeof(int), 0, cudaMemcpyDeviceToHost);

   cudaMemcpyFromSymbol(hostArray, Cglobal, N*sizeof(int));
}



int main()
{

  // Initialize the host array A and B
  int A[N]={1,2,3,4,5};
  int B[N]={5,4,3,2,1};
  int C[N];

  // Initialize array A on the device with values from hostArray
  initializeGlobal_A(A);

  // Initialize array B on the device with values from hostArray
  initializeGlobal_B(B);


  //Launch kernel with N threads (one per element in the array)
  addArray<<<1,N>>>();

  cudaDeviceSynchronize();

  //Retrieve modified globalArray from the device
  retrieveGlobalArray(C);


  //Print the modified global array
  printf("Modified globalArray: \n");
  for (int i=0; i<N; i++)
  {
    printf("%d ",C[i]);
  }

  printf("\n");


  return 0;
}

Writing assign2.cu


In [2]:
!nvcc -o exe assign2.cu
!./exe

Modified globalArray: 
6 6 6 6 6 
